In [1]:
import pandas as pd
import os


In [2]:
df = pd.read_csv("estat_lfsq_pganws_en.csv")

In [3]:
df.head()

,DATAFLOW,LAST UPDATE,freq,unit,sex,citizen,age,wstatus,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:LFSQ_PGANWS(1.0),15/03/24 11:00:00,Q,THS_PER,F,EU27_2020_FOR,Y15-19,ACT,AT,1998-Q1,NaN,u
1,ESTAT:LFSQ_PGANWS(1.0),15/03/24 11:00:00,Q,THS_PER,F,EU27_2020_FOR,Y15-19,ACT,AT,1999-Q1,NaN,u
2,ESTAT:LFSQ_PGANWS(1.0),15/03/24 11:00:00,Q,THS_PER,F,EU27_2020_FOR,Y15-19,ACT,AT,2000-Q1,NaN,u
3,ESTAT:LFSQ_PGANWS(1.0),15/03/24 11:00:00,Q,THS_PER,F,EU27_2020_FOR,Y15-19,ACT,AT,2000-Q2,NaN,u
4,ESTAT:LFSQ_PGANWS(1.0),15/03/24 11:00:00,Q,THS_PER,F,EU27_2020_FOR,Y15-19,ACT,AT,2001-Q1,NaN,u


eliminamos columans inutiles

In [4]:
df = df.drop(['DATAFLOW','freq','unit','OBS_FLAG','LAST UPDATE'],axis=1)

In [5]:
df['citizen'].unique()

array(['EU27_2020_FOR', 'FOR', 'NAT', 'NEU27_2020_FOR', 'NRP', 'STLS',
       'TOTAL'], dtype=object)

In [6]:
df['age'].unique()

array(['Y15-19', 'Y15-24', 'Y15-39', 'Y15-59', 'Y15-64', 'Y15-74',
       'Y20-24', 'Y20-64', 'Y25-29', 'Y25-49', 'Y25-54', 'Y25-59',
       'Y25-64', 'Y30-34', 'Y35-39', 'Y40-44', 'Y40-59', 'Y40-64',
       'Y45-49', 'Y50-54', 'Y50-59', 'Y50-64', 'Y55-59', 'Y55-64',
       'Y60-64', 'Y65-69', 'Y70-74', 'Y_GE15', 'Y_GE25', 'Y_GE50',
       'Y_GE65', 'Y_GE75'], dtype=object)

añdimos columnas con fechas año mes y dia

In [6]:
# Función para mapear los trimestres a meses específicos
def map_quarter_to_month(quarter):
    year, q = quarter.split('-Q')
    if q == '1':
        return f"{year}-03-01"
    elif q == '2':
        return f"{year}-06-01"
    elif q == '3':
        return f"{year}-09-01"
    elif q == '4':
        return f"{year}-12-01"
    else:
        return None

# Crear la nueva columna 'year_month' basada en la columna 'TIME_PERIOD'
df['year_month'] = df['TIME_PERIOD'].apply(map_quarter_to_month)

nos quedamos con los extranjeros activos

In [7]:
df_foreigners_act = df[(df['citizen']=='FOR') & (df['wstatus']=='ACT')]
df_foreigners_act.head(10)

,sex,citizen,age,wstatus,geo,TIME_PERIOD,OBS_VALUE,year_month
378374,F,FOR,Y15-19,ACT,AT,1998-Q1,9.5,1998-03-01
378375,F,FOR,Y15-19,ACT,AT,1999-Q1,7.3,1999-03-01
378376,F,FOR,Y15-19,ACT,AT,2000-Q1,9.7,2000-03-01
378377,F,FOR,Y15-19,ACT,AT,2000-Q2,8.8,2000-06-01
378378,F,FOR,Y15-19,ACT,AT,2001-Q1,7.7,2001-03-01
378379,F,FOR,Y15-19,ACT,AT,2002-Q1,7.6,2002-03-01
378380,F,FOR,Y15-19,ACT,AT,2003-Q1,6.5,2003-03-01
378381,F,FOR,Y15-19,ACT,AT,2003-Q2,4.6,2003-06-01
378382,F,FOR,Y15-19,ACT,AT,2003-Q3,5.6,2003-09-01
378383,F,FOR,Y15-19,ACT,AT,2003-Q4,6.2,2003-12-01


y con la fecha >= 2014

In [8]:
df_foreigners_act = df_foreigners_act[df_foreigners_act['year_month'] >= '2014-01-01']

In [9]:
df_foreigners_act.head(10)

,sex,citizen,age,wstatus,geo,TIME_PERIOD,OBS_VALUE,year_month
378424,F,FOR,Y15-19,ACT,AT,2014-Q1,7.4,2014-03-01
378425,F,FOR,Y15-19,ACT,AT,2014-Q2,6.5,2014-06-01
378426,F,FOR,Y15-19,ACT,AT,2014-Q3,6.1,2014-09-01
378427,F,FOR,Y15-19,ACT,AT,2014-Q4,7.2,2014-12-01
378428,F,FOR,Y15-19,ACT,AT,2015-Q1,5.3,2015-03-01
378429,F,FOR,Y15-19,ACT,AT,2015-Q2,5.4,2015-06-01
378430,F,FOR,Y15-19,ACT,AT,2015-Q3,6.9,2015-09-01
378431,F,FOR,Y15-19,ACT,AT,2015-Q4,7.1,2015-12-01
378432,F,FOR,Y15-19,ACT,AT,2016-Q1,7.5,2016-03-01
378433,F,FOR,Y15-19,ACT,AT,2016-Q2,10.0,2016-06-01


In [11]:
countries = ['AT', 'BE', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES',
       'FI', 'FR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LU', 'ME',
       'MT', 'NL', 'NO', 'PL', 'PT', 'RS', 'SE', 'SI', 'SK', 'UK', 'BA',
       'BG', 'LT', 'LV', 'MK', 'RO', 'TR']

creamos df por pais

In [12]:
# Crear un diccionario de DataFrames, uno por cada país
dfs_by_country = {}
for country in countries:
    dfs_by_country[country] = df_foreigners_act[df['geo'] == country]

C:\Users\ricar\AppData\Local\Temp\ipykernel_20756\1307531664.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfs_by_country[country] = df_foreigners_act[df['geo'] == country]


paises con df vacio

In [13]:
# Verificar qué países tienen DataFrame vacío
paises_vacios = [pais for pais, df_pais in dfs_by_country.items() if df_pais.empty]

print("Países con DataFrame vacío:")
print(paises_vacios)

Países con DataFrame vacío:
['TR']


lo eliminamos

In [14]:
del dfs_by_country['TR']

In [15]:
# Imprimir los DataFrames por país
for country, df_country in dfs_by_country.items():
    print(f"DataFrame para el país {country}:")
    # Agrupar por 'TIME_PERIOD' y sumar 'OBS_VALUE'
    df_country['total_obs_value'] = df_country.groupby('year_month')['OBS_VALUE'].transform('sum')

    print(df_country)

DataFrame para el país AT:
        sex citizen     age wstatus geo TIME_PERIOD  OBS_VALUE  year_month  \
378424    F     FOR  Y15-19     ACT  AT     2014-Q1        7.4  2014-03-01   
378425    F     FOR  Y15-19     ACT  AT     2014-Q2        6.5  2014-06-01   
378426    F     FOR  Y15-19     ACT  AT     2014-Q3        6.1  2014-09-01   
378427    F     FOR  Y15-19     ACT  AT     2014-Q4        7.2  2014-12-01   
378428    F     FOR  Y15-19     ACT  AT     2015-Q1        5.3  2015-03-01   
...      ..     ...     ...     ...  ..         ...        ...         ...   
5820222   T     FOR  Y_GE75     ACT  AT     2022-Q4        NaN  2022-12-01   
5820223   T     FOR  Y_GE75     ACT  AT     2023-Q1        NaN  2023-03-01   
5820224   T     FOR  Y_GE75     ACT  AT     2023-Q2        NaN  2023-06-01   
5820225   T     FOR  Y_GE75     ACT  AT     2023-Q3        NaN  2023-09-01   
5820226   T     FOR  Y_GE75     ACT  AT     2023-Q4        NaN  2023-12-01   

         total_obs_value  
378424   

C:\Users\ricar\AppData\Local\Temp\ipykernel_20756\247368248.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country['total_obs_value'] = df_country.groupby('year_month')['OBS_VALUE'].transform('sum')
C:\Users\ricar\AppData\Local\Temp\ipykernel_20756\247368248.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country['total_obs_value'] = df_country.groupby('year_month')['OBS_VALUE'].transform('sum')
C:\Users\ricar\AppData\Local\Temp\ipykernel_20756\247368248.py:5: SettingWithCopyWarning: 
A value

salvamos los dfs en csv

In [16]:
if not os.path.exists("data"):
    os.makedirs("data")

# Iterar sobre las claves y valores del diccionario
for pais, df_pais in dfs_by_country.items():
    # Generar el nombre del archivo CSV
    nombre_archivo = f"data/foreigner_act_{pais}.csv"
    # Guardar el DataFrame como un archivo CSV
    df_pais.to_csv(nombre_archivo, index=False)
